In [ ]:
from PIL import Image
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification

# Загрузка модели
processor = LayoutLMv3Processor.from_pretrained("HYPJUDY/layoutlmv3-large-finetuned-funsd")
model = LayoutLMv3ForTokenClassification.from_pretrained("HYPJUDY/layoutlmv3-large-finetuned-funsd")

In [ ]:


import torch
from PIL import Image
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification

# Load pre-trained processor and model
processor = LayoutLMv3Processor.from_pretrained("HYPJUDY/layoutlmv3-large-finetuned-funsd")
model = LayoutLMv3ForTokenClassification.from_pretrained("HYPJUDY/layoutlmv3-large-finetuned-funsd")

# Load and preprocess the image
image_path = "passport.jpg"  # Replace with your image path
image = Image.open(image_path).convert("RGB").resize((224, 224))

# Process image to get inputs (pixel values, bounding boxes, etc.)
inputs = processor(image, return_tensors="pt", truncation=True)

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions
predictions = outputs.logits.argmax(-1).squeeze().tolist()

# Map predictions to labels (using model's label map)
id2label = model.config.id2label
predicted_labels = [id2label[pred] for pred in predictions]

# Print results (simplified output)
print("Predicted labels for tokens:")
for token, label in zip(inputs["input_ids"].squeeze().tolist(), predicted_labels):
    print(f"Token ID: {token}, Label: {label}")

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
       output_dir="./results",
       learning_rate=5e-5,
       per_device_train_batch_size=4,
       num_train_epochs=3
)
train_dataset = 'data/funsd/training_data/annotations'
trainer = Trainer(
       model=model,
       args=training_args,
       train_dataset=train_dataset
   )
trainer.train()

# Интеграция NLP проверок

In [ ]:
from dateutil.parser import parse

def validate_dates(text):
    try:
        parse(text, dayfirst=True)
        return True
    except ValueError:
        return False

# В конвейере обработки
for word, label in zip(words, labels):
    if label == "DATE" and not validate_dates(word):
        print(f"Ошибка в дате: {word}")

In [5]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering", 
    model="DeepPavlov/rubert-base-cased"
)

context = "Паспорт выдан ОВД Центрального округа Москвы"
question = "Кем выдан документ?"
answer = qa_pipeline(question=question, context=context)["answer"]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use mps:0
